In [1]:
import re, json
from copy import copy

In [2]:
scenario_name = "Ghana"

with open(f"{scenario_name}.txt", "r", encoding="utf-8") as f:
    slides = f.read().split("\n===\n")

In [3]:
default_continue = "Продолжить"

In [4]:
for i in range(len(slides)):
    slides[i] = {
        "id": i,
        "metatext": slides[i].strip("\n")
    }
    stroki = slides[i]["metatext"].split("\n")
    buttons = []
    
    for stroka in stroki:
        q = re.search(" =>", stroka)
        if stroka.startswith("* ") and q:
            
            text = stroka[2 : q.start()].strip(" ")
            action = None
            destination = None
            
            dest = re.split("[/]", stroka[q.end() : ].strip(" "))
            for elem in dest:
                elem = elem.strip()
                if elem == "":
                    continue
                if elem.startswith("<<set"):
                    action = copy(elem)
                else:
                    destination = elem[0:]
            
            current_button = {"text": text}
            
            if action:
                current_button["action"] = action
            if destination:
                current_button["destination_slidename"] = destination
            
            buttons.append(current_button)
            
        elif stroka.startswith("(") and stroka.endswith(")"):
            slides[i]["slidename"] = stroka.strip("() ")
        
        elif stroka.startswith("{if "):
            stroka = re.split("[{}]", stroka[1:])
            
            conditions = []
            
            for elem in stroka:
                slovar = {}
                
                if elem.startswith("if"):
                    slovar["criterion"] = elem
                elif elem:
                    slovar["result"] = elem
                
                if slovar:
                    conditions.append(slovar)
            
            if conditions:
                slides[i]["conditions"] = conditions
        
        elif stroka.startswith("img="):
            slides[i]["img"] = re.sub(" ", "_", stroka[4:].strip("\"\«\»\„\“"))
        
        elif stroka.startswith("!"):
            slides[i]["comment"] = stroka[1:]
        
        elif stroka.startswith("#"):
            slides[i]["tag"] = stroka.strip("# ")
        
        else:
            slides[i]["text"] = stroka
    
    if buttons:
        slides[i]["choices"] = buttons
    
    del slides[i]["metatext"]
    
    if "choices" in slides[i]:
        for j in range(len(slides[i]["choices"])):
            if not "destination_slidename" in slides[i]["choices"][j]:
                slides[i]["choices"][j]["destination_id"] = i+1
    else:
        slides[i]["choices"] = [{
            "text": default_continue,
            "destination_id": i+1
        }]
    
    print(slides[i])

{'id': 0, 'img': '1.png', 'comment': 'может быть, карта?', 'text': '1957 год. Гана, бывшая британская колония в Западной Африке, которую англичане называли Золотой Берег, обрела независимость.', 'choices': [{'text': 'Продолжить', 'destination_id': 1}]}
{'id': 1, 'comment': 'показать Нкруму (на трибуне, издалека?)', 'text': 'Борьбу за деколонизацию и независимость Ганы возглавил активист и политолог Кваме Нкрума, ставший вскоре первым избранным президентом новой страны.', 'choices': [{'text': 'Продолжить', 'destination_id': 2}]}
{'id': 2, 'img': 'Ghana_Future.png', 'comment': 'фотографии, праздничные шествия в честь независимости', 'text': 'Все шесть миллионов жителей страны разных народов и религий — от мусульман в северных провинциях, близких к Сахаре, до католиков на прибрежном юге — с надеждой смотрят в будущее.', 'choices': [{'text': 'Продолжить', 'destination_id': 3}]}
{'id': 3, 'text': 'Каким будет новое ганское общество? Как будет выглядеть новая ганская культура, свободная от к

In [5]:
with open(f"{scenario_name}.json", "w", encoding="utf-8") as f:
    json.dump(slides, f, ensure_ascii=False, indent=2)